In [1]:
import random
import math

importing Jupyter notebook from WordRAM.ipynb


In [8]:
class BitVector:    
    def __init__(self, n, verbose=False):
        self.c = 4
        
        self.n = n
        self.verbose = verbose
        lgn = Reg(n).lgceil()
        assert lgn._x * lgn._x < n, "The BitVector is too small."
        
        self.bs = Mem(Reg(3), lgn) # block sizes
        self.bs[Reg(1)] = lgn * lgn # large block size
        self.bs[Reg(2)] = lgn / Reg(2) # small block size
        if verbose: print("bs: " + str(self.bs))
            
            
        self.lb = Mem(Reg(1))
        self.lb[Reg(0)] = lgn.pow(Reg(self.c)) # the length bound of select index
            
        self._bv = Mem(Reg(n), Reg(1))
        
    def _randomize(self):
        # Generate data randomly for experiments
        self._bv._randomize()
    
    def _buildIndexBF(self):
        # Build brute force index
        # Only build index for rank1, because rank0 can be easily derived from rank1
        self.indexSizeBF = 0
        self._r1bf = [] # brute force index for rank1
        crt = 0
        for i in range(self.n):
            crt += self._bv._getBit(i) == 1
            self._r1bf.append(crt)
        self.indexSizeBF += len(self._r1bf) * math.ceil(math.log(self.n + 1, 2))
        
        self._s0bf = [None for _ in range(self.n + 1)] # brute force index for select0
        self._s1bf = [None for _ in range(self.n + 1)] # brute force index for select1
        crt0 = crt1 = 1
        for i in range(self.n):
            b = self._bv._getBit(i)
            if b == 0:
                self._s0bf[crt0] = i
                crt0 += 1
            elif b == 1:
                self._s1bf[crt1] = i
                crt1 += 1
        self.indexSizeBF += len(self._s0bf) * math.ceil(math.log(self.n + 1, 2))
        self.indexSizeBF += len(self._s1bf) * math.ceil(math.log(self.n + 1, 2))
    
    
    def _buildIndexRank(self):
        # Only build index for rank1, because rank0 can be easily derived from rank1
        # allocate one more slot for better readability
        lgn = Reg(self.n).lgceil()
        self.rwl = Mem(Reg(4), lgn) # word lengths of rank index
        self.rsize = Mem(Reg(4), lgn) # size of rank index
        self.rwl[Reg(1)] = lgn # at most n ones
        self.rwl[Reg(2)] = (self.bs[Reg(1)] + Reg(1)).lgceil() # at most bs1 ones
        self.rwl[Reg(3)] = (self.bs[Reg(2)] + Reg(1)).lgceil() # at most bs2 ones
        if self.verbose: print("rwl: " + str(self.rwl))
        self.rsize[Reg(1)] = Reg(self.n).divceil(self.bs[Reg(1)])
        self.rsize[Reg(2)] = self.rsize[Reg(1)] * (self.bs[Reg(1)].divceil(self.bs[Reg(2)]))
        self.rsize[Reg(3)] = (Reg(1) << self.bs[Reg(2)]) * self.bs[Reg(2)] # at most 2 ** `bs[2]` rows and `bs[2]` columns
        if self.verbose: print("rsize: " + str(self.rsize))
        
        self._r1 = Mem(self.rsize[Reg(1)], self.rwl[Reg(1)])
        self._r2 = Mem(self.rsize[Reg(2)], self.rwl[Reg(2)])
        self._r3 = Mem(self.rsize[Reg(3)], self.rwl[Reg(3)])
        c1 = Reg(0) # c1 ones in the whole vector
        c2 = Reg(0) # c2 ones in some large block
        i = Reg(0) # i-th bit in the whole vector
        j = Reg(0) # j-th bit in some large block
        k = Reg(0) # k-th small block
        n = Reg(self.n)
        while i < n:
            if i % self.bs[Reg(1)] == Reg(0):
                self._r1[i / self.bs[Reg(1)]] = c1
                c2 = Reg(0)
                j = Reg(0)
            if j % self.bs[Reg(2)] == Reg(0):
                self._r2[k] = c2
                k += Reg(1)
            
            b = self._bv._getBit(i._x)
            if (b == 1):
                c1 += Reg(1)
                c2 += Reg(1)
            
            i += Reg(1)
            j += Reg(1)
            
        row = Reg(1) << self.bs[Reg(2)]
        column = self.bs[Reg(2)]
        i = Reg(0)
        while i < row:
            j = Reg(0)
            while j < column:
                self._r3[i * column + j] = (i >> (column - j - Reg(1))).getOnes()
                j += Reg(1)
            i += Reg(1)
        if self.verbose:
            print("r1: " + str(self._r1))
            print("r2: " + str(self._r2))
            print("r3: " + str(self._r3))
    

    class SelectBlockIndex:
        class TreeNode:
            def __init__(self, buf, lgn, l):
                
                
        
        def __init__(self, buf, l, bv):
            self.isTree = Mem(Reg(1), Reg(1))
            if (l > bv.lb[0]):
                self.isTree[Reg(0)] = Reg(0)
                self.buf = buf.clone()
            else:
                self.isTree[Reg[0]] = Reg(1)
                lgn = Reg(bv.n).lgceil()
                self.tree = TreeNode(buf, lgn, l)
            
                
    
    
    def _buildIndexSelectImpl(self, bit):
        # allocate one more slot for better readability
        lgn = Reg(self.n).lgceil()
        self.swl[bit] = Mem(Reg(2), lgn) # word lengths of select index
        self.ssize[bit] = Mem(Reg(2), lgn) # size of select index
        self.swl[bit][Reg(1)] = lgn # at most n elements
        self.ssize[bit][Reg(1)] = Reg(self.max01[bit]).divceil(lgn * lgn) # number of large blocks for select index
        self._s1[bit] = Mem(self.ssize[bit][Reg(1)], self.swl[bit][Reg(1)])
        self._s2[bit] = [None for _ in range(self.ssize[bit][Reg(1)].get())]
        lasti = Reg(0)
        i = Reg(0) # index in the whole vector
        crt = Reg(0) # index in one large block
        buf = Mem(lgn * lgn, (lgn * lgn).lgceil()) # bit buffer for one large block
        bid = Reg(0) # index in the buffer
        bfid = Reg(0) # index of the buffer
        while i < Reg(self.n):
            if self._bv._getBit(i.get()).get() == bit:
                buf[bid] = crt
                bid += Reg(1)   
            if bid == lgn * lgn:
                # buffer is filled up
                self._s1[bit][bfid] = i
                l = buf[lgn * lgn - Reg(1)] - lasti + Reg(1)
                self._s2[bit][bfid] = SelectBlockIndex(buf, l, self)
                bfid += Reg(1)
                bid = Reg(0)
                crt = Reg(0)
                lasti.set(i + Reg(1))
                
            i += Reg(1)
            crt += Reg(1) 
        
        
        
        
    
    def _buildIndexSelect(self):
        self.max01 = [self.rank0(self.n - 1).get(), self.rank1(self.n - 1).get()]
        self.swl = [None, None]
        self.ssize = [None, None]
        self._s1 = [None, None]
        self._s2 = [None, None]
        self._buildIndexSelectImpl(0)
        self._buildIndexSelectImpl(1)
        
    
    def buildIndex(self):
        self._buildIndexBF()
        self._buildIndexRank()
        self._buildIndexSelect()


        
    def rank0(self, i):
        return i + Reg(1) - self.rank1(i)
    
    def select0(self, i):
        return NotImplemented
    
    def rank1(self, i):
        bs1num = i / self.bs[Reg(1)]
        bs1start = bs1num * self.bs[Reg(1)]
        bs1end = bs1start + self.bs[Reg(1)]
        i -= bs1start
        bs2num = i / self.bs[Reg(2)]
        bs2start = bs2num * self.bs[Reg(2)]
        bs2end = bs2start + self.bs[Reg(2)]
        bs2v = self._bv._getMultiBits(bs1start + bs2start, bs1start + bs2end)
        bs2i = i - bs2start
        
        rank = Reg(0)
        rank += self._r1[bs1num]
        rank += self._r2[bs1num * (self.bs[Reg(1)].divceil(self.bs[Reg(2)])) + bs2num]
        rank += self._r3[bs2v * self.bs[Reg(2)] + bs2i]
        return rank
    
    def select1(self, i):
        return NotImplemented
    
    def rank0BF(self, i):
        return i + 1 - self.rank1BF(i)
    
    def select0BF(self, i):
        return self._s0bf[i]
    
    def rank1BF(self, i):
        return self._r1bf[i]
    
    def select1BF(self, i):
        return self._s1bf[i]
    
    def test(self, iteration):
        for _ in range(iteration):
            i = random.randrange(0, self.n)
            assert self.rank0(Reg(i)).get() == self.rank0BF(i)
            assert self.rank1(Reg(i)).get() == self.rank1BF(i)
#             assert self.select0(Reg(i)).get() == self.select0BF(i)
#             assert self.select1(Reg(i)).get() == self.select1BF(i)

In [9]:
if __name__ == '__main__':
    # Test the bit vector
#     for n in range(500, 10000, 500):
    for n in range(500, 501):
        print("n =", n)
        time = 0
        space = 0
        spaceBF = 0
#         for _ in range(10):
        for _ in range(1):
            CreateMachine(math.ceil(math.log(n, 2)) + 1, 16)
            bv = BitVector(n)
            bv._randomize()
            bv.buildIndex()
            GetMachine().totalExecution = 0
            bv.test(200)
            time += GetMachine().totalExecution
            space += GetMachine().totalMemSize - bv._bv.memSize
            spaceBF += bv.indexSizeBF
        print("time =", time)
        print("space =", space, space / n)
        print("spaceBF = ", spaceBF)
        print()

n = 500
time = 32584
space = 1383 2.766
spaceBF =  13500

